## Actividad 1 06/04/2025

#### Importación de librerias

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, regexp_replace, size
import requests

#### Creación de sesión de Spark

In [3]:
spark = SparkSession.builder.appName("Analaislibro").getOrCreate()

#### Descarga del libro desde Gutenberg

In [4]:
url = "https://www.gutenberg.org/cache/epub/67126/pg67126.txt"

In [6]:
response = requests.get(url) #Solicitud get para descargar el archivo y guardalo en response 
libro_texto = response.text #response.text contiene el contenido del archivo descargado y se guarda en libro_texto

#### Guargar el contenido del libro en un archivo local

In [7]:
with open("libro.txt", "w", encoding="utf-8") as file:
    file.write(libro_texto) #Guarda (con codificación UTF-8) el contenido descargado del libro en un archivo local llamado libro.txt

#### Crear un RDD a partir del archivo de texto descargado

In [9]:
rdd = spark.sparkContext.textFile("libro.txt") #Carga el archivo libro.txt como un RDD (Resilient Distributed Dataset). Cada línea del archivo será un elemento en el RDD.

#### Filtrar líneas vacías

In [10]:
rdd = rdd.filter(lambda line: line.strip() != "") #line.strip() elimina los espacios y != "" asegura que solo queden las líneas con contenido.

#### Filtrar los capítulos del libro

In [11]:
capitulos = rdd.filter(lambda line: line.strip().startswith("CHAPTER")).collect() #filtran las líneas que empiezan con la palabra "CHAPTER". El resultado se almacena en capitulos

#### Dividir el texto en párrafos

In [12]:
parrafos = rdd.flatMap(lambda line: line.split("\n\n")) #flatMap divide cada línea en párrafos, asumiendo que los párrafos están separados por dos saltos de línea (\n\n). flatMap aplana el resultado, generando una lista de párrafos en lugar de una lista de listas.

#### Crear un DataFrame con los párrafos

In [13]:
df_parrafos = parrafos.map(lambda p: (p,)).toDF(["parrafo"]) #Se convierte el RDD parrafos en un DataFrame, cada elemento (párrafo) será una fila. Se le da el nombre de columna "parrafo".
df_parrafos.show() #Imprime el DataFrame

+--------------------+
|             parrafo|
+--------------------+
|The Project Guten...|
|This ebook is for...|
|most other parts ...|
|whatsoever. You m...|
|of the Project Gu...|
|at www.gutenberg....|
|you will have to ...|
|before using this...|
|Title: El caballe...|
|Author: Benito Pé...|
|Release date: Jan...|
|                M...|
|   Language: Spanish|
|Original publicat...|
|Credits: Ramón Pa...|
|*** START OF THE ...|
|NOTA DE TRANSCRIP...|
|  * Las cursivas ...|
|    acotaciones e...|
|    convertido a ...|
+--------------------+
only showing top 20 rows



#### Limpiar los párrafos

In [14]:
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo"), "[^a-zA-Z\s]", "").alias("parrafo_limpio")) #Se crea una nueva columna parrafo_limpio utilizando regexp_replace. Elimina cualquier carácter no alfabético ni espacio (números, puntuación y otros caracteres no deseados).

#### Reemplazar múltiples espacios por uno solo

In [15]:
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo_limpio"), "\s+", " ").alias("parrafo_limpio")) #Se reemplazan los múltiples espacios consecutivos por un único espacio.

#### Separar los párrafos en oraciones

In [16]:
df_parrafos = df_parrafos.withColumn("oraciones", split(col("parrafo_limpio"), "\.")) #Se crea una nueva columna llamada oraciones, en la que se separan los párrafos limpios en oraciones, utilizando el punto (.) como delimitador.

#### Contar el número de oraciones por párrafo

In [17]:
df_parrafos = df_parrafos.withColumn("num_oraciones", size(col("oraciones"))) #Se calcula el número de oraciones en cada párrafo utilizando la función size(), que devuelve el tamaño de la lista generada por split().

#### Separar los párrafos en palabras

In [18]:
df_parrafos = df_parrafos.withColumn("palabras", split(col("parrafo_limpio"), " ")) #Se crea una nueva columna palabras, que contiene una lista de palabras por cada párrafo.

#### Contar el número de palabras por párrafo

In [19]:
df_parrafos = df_parrafos.withColumn("num_palabras", size(col("palabras"))) #Se calcula el número de palabras en cada párrafo con la función size().

#### Mostrar el DataFrame final

In [ ]:
df_parrafos.show() #Muestra el DataFrame final con todas las columnas.